# Unit 8.3 : SQL Case Study - Country Club

In [1]:
import sqlite3
import pandas as pd
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn, query):
    """ Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

### SQL querys test

In [3]:
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)

2.6.0


In [4]:
query_test = """
        SELECT *
        FROM FACILITIES
        """

In [5]:
with conn: 
    print("Query all tasks")
    select_all_tasks(conn, query_test)

Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [6]:
## Read sqlite query results into a pandas DataFrame
df = pd.read_sql_query(query_test, conn)

## Verify that result of SQL query is stored in the dataframe
display(df)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


### Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [7]:
query10 = '''
          SELECT Facilities.name AS Facility, 
          SUM(CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost * Bookings.slots
             ELSE Facilities.membercost* Bookings.slots END) AS total_revenue
          FROM Bookings
          LEFT JOIN Members
          USING (memid)
          LEFT JOIN Facilities
          USING (facid)
          GROUP BY Facility
          HAVING total_revenue < 1000.0
          ORDER BY total_revenue DESC
          '''

## Read sqlite query results into a pandas DataFrame
df = pd.read_sql_query(query10, conn)

## Verify that result of SQL query is stored in the dataframe
display(df)

,Facility,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [8]:
query11 = '''
        SELECT m.memid, 
               CASE WHEN m.memid = 0 THEN m.surname
                    ELSE m.surname || ", " || m.firstname END AS "Member Name",
               CASE WHEN r.memid = 0 THEN r.surname
                    ELSE r.surname || ", " || r.firstname END AS "Recommended By"
        FROM Members AS m
        LEFT JOIN Members AS r
        ON m.recommendedby = r.memid
        WHERE m.memid <>0
        ORDER BY "Member Name", "Recommended By"
        
        '''

## Read sqlite query results into a pandas DataFrame
df = pd.read_sql_query(query11, conn)

## Verify that result of SQL query is stored in the dataframe
display(df)

,memid,Member Name,Recommended By
0,15,"Bader, Florence","Stibbons, Ponder"
1,12,"Baker, Anne","Stibbons, Ponder"
2,16,"Baker, Timothy","Farrell, Jemima"
3,8,"Boothe, Tim","Rownam, Tim"
4,5,"Butters, Gerald","Smith, Darren"
5,22,"Coplin, Joan","Baker, Timothy"
6,36,"Crumpet, Erica","Smith, Tracy"
7,7,"Dare, Nancy","Joplette, Janice"
8,28,"Farrell, David",None
9,13,"Farrell, Jemima",None


### Q12: Find the facilities with their usage by member, but not guests

In [9]:
query12 = '''
        SELECT memid, "Member Name", Facilities.name AS Facility, "Total Usage"
        FROM (SELECT memid, facid, SUM(slots) AS "Total Usage" 
              FROM Bookings
              WHERE memid NOT IN (0)
              GROUP BY memid, facid
              ) AS m_usage_count
        LEFT JOIN (SELECT memid, surname || ", " || firstname AS "Member Name"
              FROM Members
              WHERE memid NOT IN (0)
              ) AS subquery2
        USING (memid)
        LEFT JOIN Facilities
        USING (facid)
        ORDER BY "Member Name", Facility
        '''

## Read sqlite query results into a pandas DataFrame
df = pd.read_sql_query(query12, conn)

## Verify that result of SQL query is stored in the dataframe
display(df)

,memid,Member Name,Facility,Total Usage
0,15,"Bader, Florence",Badminton Court,27
1,15,"Bader, Florence",Massage Room 2,4
2,15,"Bader, Florence",Pool Table,23
3,15,"Bader, Florence",Snooker Table,66
4,15,"Bader, Florence",Squash Court,4
...,...,...,...,...
197,33,"Tupperware, Hyacinth",Squash Court,2
198,29,"Worthington-Smyth, Henry",Badminton Court,15
199,29,"Worthington-Smyth, Henry",Massage Room 1,2
200,29,"Worthington-Smyth, Henry",Pool Table,37


### Q13: Find the facilities usage by month, but not guests

In [10]:
query13 = '''
          SELECT name AS Facility, Month, "Usage of the Month"
          FROM Facilities
          LEFT JOIN (SELECT facid, strftime('%m', starttime) AS Month,  
                            SUM(slots) AS "Usage of the Month"
                     FROM Bookings
                     WHERE memid <> 0
                     GROUP BY facid, Month
                     ) AS subquery
          USING (facid)
          ORDER BY Facility, Month
          '''
                     
## Read sqlite query results into a pandas DataFrame
df = pd.read_sql_query(query13, conn)

## Verify that result of SQL query is stored in the dataframe
display(df)

,Facility,Month,Usage of the Month
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
